In [ ]:
import smtplib
import schedule
import time
from email.mime.text import MIMEText
from email.header import Header
from email.utils import formataddr

# ================= 配置区域 =================
USER_CONFIG = {
    'smtp_server': 'smtp.163.com',
    'smtp_port': 465,               # 网易邮箱SSL端口
    'sender_email': 'pkuaft@163.com',    # 你的163邮箱账号
    'auth_code': 'YPxa2E3YdAVrCy66',          # 【重要】这里填刚才获取的授权码
    'receiver_email': 'oudaisong2003@163.com'  # 接收邮件的人
}
# ===========================================

def send_daily_report():
    print("⏳ 正在尝试发送邮件...")
    try:
        # 1. 构建邮件对象
        # 参数1: 正文内容, 参数2: 格式(plain/html), 参数3: 编码
        content = "您好，这是基于Python定时任务自动发送的测试邮件。\n\n系统运行状态：正常"
        msg = MIMEText(content, 'plain', 'utf-8')
        
        # 2. 设置邮件头 (防止被当做垃圾邮件，格式很重要)
        # formataddr((显示昵称, 邮箱地址))
        msg['From'] = formataddr((Header("定时助手", 'utf-8').encode(), USER_CONFIG['sender_email']))
        msg['To'] = formataddr((Header("管理员", 'utf-8').encode(), USER_CONFIG['receiver_email']))
        msg['Subject'] = Header("【系统通知】每日定时汇报", 'utf-8')

        # 3. 连接网易服务器 (使用 SSL 加密)
        server = smtplib.SMTP_SSL(USER_CONFIG['smtp_server'], USER_CONFIG['smtp_port'])
        
        # 4. 登录并发送
        server.login(USER_CONFIG['sender_email'], USER_CONFIG['auth_code'])
        server.sendmail(USER_CONFIG['sender_email'], [USER_CONFIG['receiver_email']], msg.as_string())
        
        # 5. 退出
        server.quit()
        print(f"✅ 邮件已成功发送至 {USER_CONFIG['receiver_email']} (时间: {time.ctime()})")

    except smtplib.SMTPAuthenticationError:
        print("❌ 认证失败：请检查你的【授权码】是否正确，不要使用登录密码。")
    except Exception as e:
        print(f"❌ 发送失败，错误信息: {e}")

# ================= 定时调度 =================
def run_scheduler():
    # 设定任务：例如每隔 1 分钟发送一次（测试用）
    # 实际使用可改为 schedule.every().day.at("09:00").do(send_daily_report)
    schedule.every(1).minutes.do(send_daily_report)
    
    print("🚀 网易邮箱定时系统已启动...")
    
    # 立即执行一次以便测试配置是否正确
    send_daily_report() 
    
    while True:
        schedule.run_pending()
        time.sleep(1)

if __name__ == "__main__":
    run_scheduler()

In [1]:
!pip install schedule